In [1]:
import sys
import os
import shutil

In [2]:
sys.path.append('..')

In [3]:
from jamo import j2hcj, h2j
import pandas as pd
import numpy as np
from konlpy.tag import Twitter

from sklearn.metrics import accuracy_score, f1_score
twitter = Twitter()

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/konlpy/tag/_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [4]:
from utils.prepare_data import *
import time
from utils.model_helper import *

/Users/1003874/anaconda/envs/lstm/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
from custum_dataset import digi_data_load, testcase_add_data_load, testcase_shuffle_data_load
from attn_bi_lstm import ABLSTM

In [6]:
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = digi_data_load()
#TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_add_data_load()
TRAIN_DOC, TRAIN_LABEL, TEST_DOC, TEST_LABEL, cols = testcase_shuffle_data_load()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29769 entries, 1 to 38319
Data columns (total 4 columns):
sent          29769 non-null object
nlu_result    29769 non-null object
class         29769 non-null object
clean_text    29769 non-null object
dtypes: object(4)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 28516 entries, 1 to 38319
Data columns (total 4 columns):
sent          28516 non-null object
nlu_result    28516 non-null object
class         28516 non-null object
clean_text    28516 non-null object
dtypes: object(4)
memory usage: 1.1+ MB
32760 1062


In [7]:
train_intent = np.apply_along_axis(np.argmax,1,TRAIN_LABEL)
test_intent = np.apply_along_axis(np.argmax,1,TEST_LABEL)

In [8]:
%%time
train_doc = TRAIN_DOC.tolist()
test_doc = TEST_DOC.tolist()

#train_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), train_doc)))
#test_doc = pd.Series(list(map(lambda x: ' '.join(twitter.morphs(x)), test_doc)))
train_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), train_doc)))
test_doc = pd.Series(list(map(lambda x: ' '.join(list(x.replace(' ','ⓢ'))), test_doc)))

CPU times: user 79.8 ms, sys: 2.13 ms, total: 81.9 ms
Wall time: 80.6 ms


In [9]:
y_train = pd.Series(train_intent)
y_test = pd.Series(test_intent)

In [10]:
x_train, x_test, tokenizer = \
        data_preprocessing_v2(train_doc, test_doc, max_len=64)
vocab_size = tokenizer.num_words

In [11]:
tokenizer.word_docs

{'3': 685,
 '요': 15349,
 '권': 1794,
 '후': 542,
 '2': 1100,
 '있': 2977,
 '5': 745,
 '8': 399,
 '구': 2964,
 '까': 1327,
 '매': 2253,
 '결': 1892,
 '할': 2177,
 '수': 2284,
 '을': 3239,
 '이': 9115,
 '월': 432,
 '귄': 5,
 'ⓢ': 26783,
 '만': 1215,
 '용': 3851,
 '재': 738,
 '천': 2352,
 '4': 587,
 '일': 2371,
 '마': 1248,
 '렴': 539,
 '트': 1888,
 '한': 3149,
 '보': 1968,
 '가': 7549,
 '다': 5189,
 '저': 2029,
 '년': 80,
 '잇': 135,
 '역': 272,
 '나': 5427,
 '확': 846,
 '0': 1225,
 '내': 1418,
 '6': 421,
 '인': 4259,
 '1': 2240,
 '사': 3240,
 '니': 4511,
 '바': 1192,
 '주': 4649,
 '세': 2803,
 '알': 1139,
 '장': 1484,
 '법': 703,
 '려': 2541,
 '영': 257,
 '아': 4470,
 '브': 168,
 '화': 1088,
 '피': 343,
 '예': 352,
 '데': 6035,
 '공': 314,
 '제': 5182,
 '정': 1351,
 '항': 130,
 '께': 276,
 '자': 1617,
 '십': 280,
 '티': 545,
 '동': 560,
 '버': 842,
 '연': 872,
 '멤': 456,
 '계': 511,
 'n': 903,
 '를': 1634,
 '서': 3707,
 '해': 5337,
 '스': 1565,
 '통': 463,
 'u': 201,
 '비': 1419,
 'g': 311,
 '환': 2477,
 '불': 2077,
 '컴': 176,
 '웰': 112,
 '딜': 143,
 '받':

In [13]:
x_dev = x_test
y_dev = y_test
dev_size = len(x_dev)
test_size = len(x_test)
print("Validation Size: ", dev_size)

config = {
    "max_len": 64,
    "hidden_size": 64,
    "vocab_size": vocab_size,
    "embedding_size": 128,
    "n_class": 33,
    "learning_rate": 0.00012, #1e-3,
    "decay": 0.90,
    "batch_size": 128,
    "train_epoch": 20
}


Validation Size:  1062


In [14]:
modelname = 'attn_bilstm_test'

try:
    shutil.rmtree('./output/'+modelname)
except OSError as e:
    if e.errno == 2:
        # 파일이나 디렉토리가 없음!
        print ('No such file or directory to remove')
        pass
    else:
        raise

In [15]:
sess = tf.Session()

classifier = ABLSTM(sess, config)
classifier.build_graph(modelname)

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

resume = True
if resume:
    saver.restore(sess, "./Saver/{}/{}.ckpt".format(modelname, modelname))

dev_batch = (x_dev, y_dev)
start = time.time()
step = 0
lr = config['learning_rate']
for e in range(config["train_epoch"]):
    
    t0 = time.time()
    print("Epoch %d start !" % (e + 1))
    for x_batch, y_batch in fill_feed_dict(x_train, y_train, config["batch_size"]):
        return_dict = run_train_step(classifier, sess, (x_batch, y_batch),lr)
        attn = get_attn_weight(classifier, sess, (x_batch, y_batch), lr)
        # plot the attention weight
        # print(np.reshape(attn, (config["batch_size"], config["max_len"])))
        if step % 10 == 0:
            feed_dict = make_test_feed_dict(classifier, (x_batch, y_batch))
            tr_merged = sess.run(classifier.merge, feed_dict)
            feed_dict = make_test_feed_dict(classifier, (x_test, y_test))
            ts_merged = sess.run(classifier.merge, feed_dict)
            classifier.train_writer.add_summary(tr_merged, step)
            classifier.test_writer.add_summary(ts_merged, step)
            
        step += 1
    t1 = time.time()
    if e+1 % 10: 
        lr = lr*config['decay']
    print("Train Epoch time:  %.3f s" % (t1 - t0))
    dev_acc = run_eval_step(classifier, sess, dev_batch)
    print("validation accuracy: %.3f " % dev_acc)

building graph
graph built successfully!
INFO:tensorflow:Restoring parameters from ./Saver/attn_bilstm_test/attn_bilstm_test.ckpt
Epoch 1 start !
Train Epoch time:  67.406 s
validation accuracy: 0.833 
Epoch 2 start !
Train Epoch time:  66.188 s
validation accuracy: 0.835 
Epoch 3 start !
Train Epoch time:  63.609 s
validation accuracy: 0.833 
Epoch 4 start !
Train Epoch time:  61.941 s
validation accuracy: 0.834 
Epoch 5 start !
Train Epoch time:  65.110 s
validation accuracy: 0.833 
Epoch 6 start !
Train Epoch time:  64.087 s
validation accuracy: 0.832 
Epoch 7 start !
Train Epoch time:  63.833 s
validation accuracy: 0.838 
Epoch 8 start !
Train Epoch time:  65.455 s
validation accuracy: 0.831 
Epoch 9 start !
Train Epoch time:  59.668 s
validation accuracy: 0.832 
Epoch 10 start !
Train Epoch time:  65.735 s
validation accuracy: 0.834 
Epoch 11 start !
Train Epoch time:  65.704 s
validation accuracy: 0.834 
Epoch 12 start !
Train Epoch time:  64.190 s
validation accuracy: 0.834 
Epo

In [16]:
if "Saver" not in os.listdir("./"):
    os.makedirs("./Saver")
if modelname not in os.listdir("./Saver"):
    os.makedirs("./Saver/{}".format(modelname))
saver.save(sess, "./Saver/{}/{}.ckpt".format(modelname, modelname))

'./Saver/attn_bilstm_test/attn_bilstm_test.ckpt'

In [17]:
print("Training finished, time consumed : ", time.time() - start, " s")
print("Start evaluating:  \n")
cnt = 0
test_acc = 0
for x_batch, y_batch in fill_feed_dict(x_test, y_test, config["batch_size"]):
    acc = run_eval_step(classifier, sess, (x_batch, y_batch))
    test_acc += acc
    cnt += 1

print("Test accuracy : %f %%" % (test_acc / cnt * 100))

Training finished, time consumed :  1995.0089662075043  s
Start evaluating:  

Test accuracy : 83.203125 %
